In [1]:
using PyCall
unshift!(PyVector(pyimport("sys")["path"]), "");
@pyimport Compute_Jacobian_Anaheim

In [2]:
numNodes = Compute_Jacobian_Anaheim.numNodes;
numLinks = Compute_Jacobian_Anaheim.numLinks;
numODpairs = Compute_Jacobian_Anaheim.numODpairs;
numZones = Compute_Jacobian_Anaheim.numZones;
od_pairs = Compute_Jacobian_Anaheim.od_pairs;
link_list_js = Compute_Jacobian_Anaheim.link_list_js;
link_length_list = Compute_Jacobian_Anaheim.link_length_list;

In [3]:
include("../Julia_files/initia_data.jl");
include("../Julia_files/load_network_uni_class.jl");
include("prepare_data_Anaheim.jl");
include("extract_data_Anaheim.jl");
include("../Julia_files/tap_MSA.jl");
include("../Julia_files/inverseVI.jl");
include("../Julia_files/demands_adjustment_gradi.jl");

In [4]:
demandsDict

Dict{Any,Any} with 2 entries:
  0 => Dict{Any,Any}(Pair{Any,Any}((8,23),1.0),Pair{Any,Any}((21,13),21.9),Pair…
  1 => Dict{Any,Any}(Pair{Any,Any}((8,23),1.02311),Pair{Any,Any}((21,13),19.786…

In [5]:
demandsDiffDict[1] = norm(demandsDicToVec(demandsDict[1]) - demandsDicToVec(demandsDict[0]))/
                     norm(demandsDicToVec(demandsDict[0]));

In [6]:
fcoeffs = [1, 0, 0, 0, .15];

# get observed flow vector (corresponding to ground truth demands and ground truth costs)
tapFlowDicDict[0], tapFlowVecDict[0] = tapMSA(demandsDict[0], fcoeffs);

In [7]:
# get arcs data corresponding to ground truth demands and flows
arcsDict[0] = observFlow("./benchmark_data/Anaheim_net.txt", tapFlowDicDict[0])

Dict{Any,Any} with 914 entries:
  (71,255)  => Arc(71,255,1800.0,0.5,2256.86)
  (323,324) => Arc(323,324,5400.0,0.5,1101.68)
  (354,353) => Arc(354,353,5400.0,1.0,8.1826)
  (60,230)  => Arc(60,230,9000.0,0.333333,2184.31)
  (223,346) => Arc(223,346,1800.0,0.5,1407.46)
  (147,146) => Arc(147,146,7200.0,0.763321,3563.88)
  (337,33)  => Arc(337,33,12600.0,0.149068,6581.72)
  (366,365) => Arc(366,365,5400.0,0.5,1008.7)
  (390,375) => Arc(390,375,5400.0,2.0,0.0)
  (315,316) => Arc(315,316,5400.0,2.57992,0.0)
  (198,197) => Arc(198,197,7200.0,0.152623,6072.44)
  (333,334) => Arc(333,334,5400.0,0.420076,1501.87)
  (367,368) => Arc(367,368,5400.0,0.720076,132.977)
  (244,339) => Arc(244,339,1800.0,0.5,1770.03)
  (274,41)  => Arc(274,41,5400.0,1.0,0.0)
  (225,330) => Arc(225,330,1800.0,0.5,997.887)
  (49,369)  => Arc(49,369,5400.0,1.0,0.0)
  (274,275) => Arc(274,275,5400.0,2.0,501.6)
  (15,254)  => Arc(15,254,5400.0,1.0,407.1)
  (226,225) => Arc(226,225,9000.0,0.937836,5169.63)
  (218,392) => A

In [8]:
deg = 6
c = 3.5
lam = 1.

coeffs_dict[(deg, c, lam, 1)], objInvVIDict[1] = train(lam, deg, c, demandsDict[1], arcsDict[0]);

In [9]:
# get initial flow vector (corresponding to initial demands)
tapFlowDicDict[1], tapFlowVecDict[1] = tapMSA(demandsDict[1], coeffs_dict[(deg, c, lam, 1)]);

demandsVecDict[0] = demandsDicToVec(demandsDict[0]);

In [10]:
# weight parameters
gamma1 = 1
gamma2 = 1

1

In [11]:
demandsVecDict[1] = demandsDicToVec(demandsDict[1]);
objFunDict[1] = objF(gamma1, gamma2, demandsVecDict[1], demandsVecDict[1], coeffs_dict[(deg, c, lam, 1)]);

In [12]:
# get arcs data corresponding to initial demands and flows
arcsDict[1] = observFlow("./benchmark_data/Anaheim_net.txt", tapFlowDicDict[1]);

In [13]:
linkCostDicDict[1] = tapFlowVecToLinkCostDict(tapFlowVecDict[1], coeffs_dict[(deg, c, lam, 1)]);

In [14]:
linkCostDicDict[1]["0"], link_length_list[1]

(1.1481580816931098,5280.0)

In [15]:
jacobiSpiessDict[1] = Compute_Jacobian_Anaheim.jacobianSpiess(numNodes, numLinks, numODpairs, od_pairs, 
                                              link_list_js, [linkCostDicDict[1]["$(i)"] for i=0:numLinks-1]);

In [16]:
# maximum number of iterations
N = 100;  

# Armijo rule parameters
rho = 2;
M = 10;

# search direction parameter
epsilon_1 = 0;

# stop criterion parameter
epsilon_2 = 1e-20;

In [17]:
for l = 1:N
    
    jacobDict[l] = jacobiSpiessDict[l]
    
    demandsVecDict[l] = demandsDicToVec(demandsDict[l]);

    descDirecDict[l] = descDirec(gamma1, gamma2, demandsVecDict[l], demandsVecDict[1], tapFlowVecDict[l], 
        tapFlowVecDict[0], jacobDict[l], numODpairs, numLinks);

    searchDirecDict[l] = searchDirec(demandsVecDict[l], descDirecDict[l], epsilon_1);

    thetaMaxDict[l] = thetaMax(demandsVecDict[l], searchDirecDict[l]);

    demandsVecDict[l+1] = similar(demandsVecDict[0]);
    
    demandsVecDict[l+1], objFunDict[l+1] = armijo(gamma1, gamma2, objFunDict[l], demandsVecDict[l], 
        demandsVecDict[1], coeffs_dict[(deg, c, lam, l)], searchDirecDict[l], thetaMaxDict[l], rho, M);
            
    demandsDict[l+1] = demandsVecToDic(demandsVecDict[l+1]);
    
    tapFlowDicDict[l+1], tapFlowVecDict[l+1] = tapMSA(demandsDict[l+1], coeffs_dict[(deg, c, lam, l)]);
        
    arcsDict[l+1] = observFlow("./benchmark_data/Anaheim_net.txt", tapFlowDicDict[l+1]);
        
#     coeffs_dict[(deg, c, lam, l+1)], objInvVIDict[l+1] = train(lam, deg, c, demandsDict[l+1], arcsDict[l+1]);
    coeffs_dict[(deg, c, lam, l+1)], objInvVIDict[l+1] = train(lam, deg, c, demandsDict[l+1], arcsDict[0]);
    
    objFunTmp = objF(gamma1, gamma2, demandsVecDict[l+1], demandsVecDict[1], 
        coeffs_dict[(deg, c, lam, l+1)])
    
    if objFunTmp > objFunDict[l+1]
        coeffs_dict[(deg, c, lam, l+1)] = coeffs_dict[(deg, c, lam, l)]
    end

    linkCostDicDict[l+1] = tapFlowVecToLinkCostDict(tapFlowVecDict[l+1], coeffs_dict[(deg, c, lam, l+1)]);
    
    jacobiSpiessDict[l+1] = Compute_Jacobian_Anaheim.jacobianSpiess(numNodes, numLinks, numODpairs, od_pairs, 
                                              link_list_js, [linkCostDicDict[l+1]["$(i)"] for i=0:numLinks-1]);
        
    demandsDiffDict[l+1] = norm(demandsVecDict[l+1] - demandsVecDict[0]) / norm(demandsVecDict[0]);
    
    # stopping criterion
    if (objFunDict[l] - objFunDict[l+1]) / objFunDict[1] < epsilon_2
        break
    end
        
end

In [18]:
# update plots based on convergence rate
N = length(objFunDict)

# normalize objective function value
for l = 1:N
    norObjFunDict[l] = objFunDict[l] / objFunDict[1];
end

In [19]:
demandsVecDict

Dict{Any,Any} with 12 entries:
  2  => [1266.01,406.857,899.682,338.577,573.642,400.476,1.9089,60.3611,71.5597…
  11 => [1267.14,407.145,899.831,338.338,572.739,400.145,1.31801,59.9214,71.256…
  0  => [1365.9,407.4,861.4,354.4,545.1,431.5,1.0,56.8,75.3,1.0  …  18.6,27.2,5…
  7  => [1266.86,407.041,899.747,338.256,572.834,400.115,1.34533,59.9354,71.274…
  9  => [1267.14,407.145,899.831,338.338,572.739,400.145,1.31801,59.9214,71.256…
  10 => [1267.14,407.145,899.831,338.338,572.739,400.145,1.31801,59.9214,71.256…
  8  => [1267.03,407.102,899.796,338.302,572.773,400.132,1.32756,59.9255,71.262…
  6  => [1266.86,407.04,899.746,338.255,572.834,400.115,1.3456,59.9356,71.2746,…
  4  => [1266.44,406.923,899.666,338.303,573.16,400.213,1.54192,60.0781,71.3732…
  3  => [1266.4,406.913,899.659,338.31,573.194,400.224,1.56208,60.0928,71.3834,…
  5  => [1266.85,407.037,899.743,338.253,572.838,400.114,1.34706,59.9365,71.275…
  1  => [1262.83,404.798,898.287,334.769,572.6,399.163,1.0448,59.471,70.7987,1

In [20]:
norObjFunDict

Dict{Any,Any} with 11 entries:
  2  => 0.661501
  11 => 0.406698
  7  => 0.431381
  9  => 0.406858
  10 => 0.406698
  8  => 0.419248
  6  => 0.431552
  4  => 0.517747
  3  => 0.528079
  5  => 0.456984
  1  => 1.0

In [21]:
coeffs_dict

Dict{Any,Any} with 11 entries:
  (6,3.5,1.0,10) => [1.0,0.00771044,-0.0513166,0.0980402,0.078622,0.0159745,0.0…
  (6,3.5,1.0,9)  => [1.0,0.00771044,-0.0513166,0.0980402,0.078622,0.0159745,0.0…
  (6,3.5,1.0,2)  => [1.0,0.00854937,-0.0530493,0.0917749,0.0837169,0.015797,0.0…
  (6,3.5,1.0,7)  => [1.0,0.0077176,-0.0513564,0.0981201,0.0785541,0.0159982,0.0…
  (6,3.5,1.0,4)  => [1.0,0.00854937,-0.0530493,0.0917749,0.0837169,0.015797,0.0…
  (6,3.5,1.0,3)  => [1.0,0.00854937,-0.0530493,0.0917749,0.0837169,0.015797,0.0…
  (6,3.5,1.0,6)  => [1.0,0.00771767,-0.0513568,0.0981208,0.0785535,0.0159984,0.…
  (6,3.5,1.0,1)  => [1.0,0.00854937,-0.0530493,0.0917749,0.0837169,0.015797,0.0…
  (6,3.5,1.0,8)  => [1.0,0.00771203,-0.0513258,0.0980602,0.0786034,0.0159818,0.…
  (6,3.5,1.0,5)  => [1.0,0.00771791,-0.0513581,0.0981236,0.078551,0.0159994,0.0…
  (6,3.5,1.0,11) => [1.0,0.00771044,-0.0513166,0.0980402,0.078622,0.0159745,0.0…

In [22]:
outfile = open("./results/norObjFunDict_$(gamma1)_$(gamma2).json", "w")

JSON.print(outfile, norObjFunDict)

close(outfile)

In [23]:
outfile = open("./results/demandsDiffDict_$(gamma1)_$(gamma2).json", "w")

JSON.print(outfile, demandsDiffDict)

close(outfile)

In [24]:
outfile = open("./results/objInvVIDict_$(gamma1)_$(gamma2).json", "w")

JSON.print(outfile, objInvVIDict)

close(outfile)

In [25]:
outfile = open("./results/coeffs_dict_$(gamma1)_$(gamma2).json", "w")

JSON.print(outfile, coeffs_dict)

close(outfile)

In [26]:
outfile = open("./results/demandsVecDict_$(gamma1)_$(gamma2).json", "w")

JSON.print(outfile, demandsVecDict)

close(outfile)

In [27]:
outfile = open("./results/demandsDict_$(gamma1)_$(gamma2).json", "w")

JSON.print(outfile, demandsDict)

close(outfile)

In [28]:
outfile = open("./results/tapFlowDicDict_$(gamma1)_$(gamma2).json", "w")

JSON.print(outfile, tapFlowDicDict)

close(outfile)

In [29]:
outfile = open("./results/tapFlowVecDict_$(gamma1)_$(gamma2).json", "w")

JSON.print(outfile, tapFlowVecDict)

close(outfile)